# Material Science repo
This is a demo notebook for "Discovery of Reticular Materials​  for Carbon Dioxide Capture​ Using GFlowNets​"

https://arxiv.org/abs/2310.07671

https://github.com/Mastro-Geppetto/matgfn

# Setup matgfn

In [2]:
!python --version

Python 3.11.12


In [3]:
!git clone https://github.com/Mastro-Geppetto/matgfn.git

Cloning into 'matgfn'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 251 (delta 8), reused 235 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (251/251), 24.74 MiB | 14.89 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [4]:
!pip install -e matgfn

Obtaining file:///content/matgfn
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 15.5 MB/s e

### restart runtime for setup to continue

In [1]:
import matplotlib,mpl_toolkits

In [2]:
import torch

In [3]:
torch.__version__

'2.1.0+cu121'

In [4]:
import matgfn

## Validate libs can be imported without errors

In [5]:
# GFlowNet
from matgfn.gflow.environments.sequence import SequenceEnvironment
from matgfn.gflow.agent import TrajectoryBalanceGFlowNet
from matgfn.gflow.flow_models.lstm import LSTM

# MOF
from matgfn.reticular import PormakeStructureBuilder
import nglview as nv

# Utils
from tqdm import tqdm
import math
import subprocess
import os
import tempfile

### add Jypter enable custom widget manager for pymatgen

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

## python lib : PormakeStructureBuilder : We can make random MOF using Pormake.
`PormakeStructureBuilder` is our class. It hides pormake instance.

`PormakerStructureBuilder` uses `MOFMask` which holds data about topology (num of node edge etc).

In [7]:
builder = PormakeStructureBuilder(topology_string="ffc", include_edges=True, block_rmsd_cutoff=0.1)
cutoff = 5000
print(f"Number of potential materials with allowed building blocks for \"FFC\" topology: {math.prod([len(x) for x in builder.mask.forward_actions_at_each_slot]):,}")

/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)


Number of potential materials with allowed building blocks: 12,789,603,200


### python lib : Pymatgen demo : Visualize a random structure using Pymatgen

In [9]:
# Make a random material
sequence = builder.random_sequence()
structure =  builder.make_structure(sequence)

# Visualise it with Pymatgen
nv.show_pymatgen(structure)

/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)


NGLWidget()

## external lib installation : zeo++
zeo++ is an open source software for performing high-throughput geometry-based analysis of porous materials and their voids.

https://www.zeoplusplus.org/about.html

In [18]:
!wget http://www.zeoplusplus.org/zeo++-0.3.tar.gz

--2025-05-11 10:02:44--  http://www.zeoplusplus.org/zeo++-0.3.tar.gz
Resolving www.zeoplusplus.org (www.zeoplusplus.org)... 204.44.192.67
Connecting to www.zeoplusplus.org (www.zeoplusplus.org)|204.44.192.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4831470 (4.6M) [application/x-gzip]
Saving to: ‘zeo++-0.3.tar.gz’

zeo++-0.3.tar.gz    100%[===================>]   4.61M  --.-KB/s    in 0.08s   

2025-05-11 10:02:45 (56.2 MB/s) - ‘zeo++-0.3.tar.gz’ saved [4831470/4831470]



### Extract and make

In [19]:
!tar -xzvf zeo++-0.3.tar.gz

zeo++-0.3/
zeo++-0.3/channel.h
zeo++-0.3/networkstorage.h
zeo++-0.3/cycle.cc
zeo++-0.3/networkaccessibility.h
zeo++-0.3/poreinfo.cc
zeo++-0.3/general.h
zeo++-0.3/Makefile.dep
zeo++-0.3/networkinfo.cc
zeo++-0.3/sphere_approx.cc
zeo++-0.3/tdpldhistogram.cc
zeo++-0.3/arguments.h
zeo++-0.3/geometry.h
zeo++-0.3/wget.csh
zeo++-0.3/string_additions.cc
zeo++-0.3/octree.hpp
zeo++-0.3/voronoicell.h
zeo++-0.3/instructions.cc
zeo++-0.3/symmetry.h
zeo++-0.3/segment.h
zeo++-0.3/heap.h
zeo++-0.3/poreinfo.h
zeo++-0.3/ray.cc
zeo++-0.3/symbcalc.h
zeo++-0.3/molecule_to_abstract.cc
zeo++-0.3/Makefile
zeo++-0.3/arguments.cc
zeo++-0.3/holograms.cc
zeo++-0.3/Docs/
zeo++-0.3/Docs/Zeo_tutorial_word.ppt
zeo++-0.3/Docs/Visualizing_pore_size_distribution_files_in_VisIt.ppt
zeo++-0.3/networkio.cc
zeo++-0.3/graphstorage.cc
zeo++-0.3/builder_examples/
zeo++-0.3/builder_examples/building_blocks/
zeo++-0.3/builder_examples/building_blocks/trigonal.xyz
zeo++-0.3/builder_examples/building_blocks/pyr_linker_without_sites

In [20]:
!cd zeo++-0.3/voro++/src && make

g++ -Wall -ansi -pedantic -O3 -c cell.cc
cell.cc: In member function ‘void voro::voronoicell_base::add_memory_vorder(vc_class&)’:
cell.cc:231:9: warning: this ‘for’ clause does not guard... []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wmisleading-indentation-Wmisleading-indentation]8;;]
  231 |         for(j=0;j<current_vertex_order;j++) p1[j]=mem[j];while(j<i) p1[j++]=0;
      |         ^~~
cell.cc:231:58: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
  231 |         for(j=0;j<current_vertex_order;j++) p1[j]=mem[j];while(j<i) p1[j++]=0;
      |                                                          ^~~~~
cell.cc:237:9: warning: this ‘for’ clause does not guard... []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wmisleading-indentation-Wmisleading-indentation]8;;]
  237 |         for(j=0;j<current_vertex_order;j++) p1[j]=mec[j];while(j<i) p1[j++]=0;
      |         ^~~
cell.cc:237:58

In [21]:
!cd zeo++-0.3/ && make

g++ -Ivoro++/src  -g  -c network.cc
In file included from Eigen/Core:263,
                 from Eigen/Dense:1,
                 from OMS.h:22,
                 from network.cc:21:
Eigen/src/Core/arch/SSE/PacketMath.h:29:39: warning: ignoring attributes on template argument ‘__m128’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wignored-attributes-Wignored-attributes]8;;]
   29 | template<> struct is_arithmetic<__m128>  { enum { value = true }; };
      |                                       ^
Eigen/src/Core/arch/SSE/PacketMath.h:30:40: warning: ignoring attributes on template argument ‘__m128i’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wignored-attributes-Wignored-attributes]8;;]
   30 | template<> struct is_arithmetic<__m128i> { enum { value = true }; };
      |                                        ^
Eigen/src/Core/arch/SSE/PacketMath.h:31:40: warning: ignoring attributes on template argument ‘__m128d’ []8;;https://gcc.gnu.org/

In [22]:
!zeo++-0.3/network

Network commandline invocation syntax:

./network [-cssr [outputfile_cssr]] 
          [-cif  [outputfile_cif]] 
          [-v1   [outputfile_v1]] 
          [-xyz  [outputfile_xyz]] 
          [-superxyz  [outputfile_xyz]] 
          [-vtk  [outputfile_vtk]] 
          [-vis  [outputfile_vtk/xyz]] 
          [-nt2  [outputfile_nt2]] 
          [-mopac  [outputfile_mop]] 
          [-supermopac  [outputfile_mop]] 
          [-res  [outputfile_res]] 
          [-zvis [outputfile_zvis]] 
          [-axs probe_radius [outputfile_axs]] 
          [-sa chan_radius probe_radius num_samples [outputfile_sa]] 
          [-vol chan_radius probe_radius num_samples [outputfile_vol]] 
          [-volpo chan_radius probe_radius num_samples [outputfile_vol]] 
          [-psd chan_radius probe_radius num_samples [outputfile_sa]] 
          [-ray chan_radius probe_radius num_samples [outputfile_ray]] 
          [-chan probe_radius [outputfile_chan]] 
          [-visVoro probe_radius [unit_cell_shifts: 

# Retricular material generation

## Our Reward function
It depends on both Gravimetric (GSA) and Volumetric (VA) surface areas

### Reward function uses zeo++ : validating zeo++ works

In [35]:
mof = builder.make_pormake_mof(sequence)
mof.write_cif("test.cif")
!zeo++-0.3/network -sa 1.525 1.525 2000 test.cif > test.sa
!zeo++-0.3/network -vol 1.525 1.525 2000 test.cif > test.vol

/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)


In [36]:
!ls -l test.*

-rw-r--r-- 1 root root 18663 May 11 10:07 test.cif
-rw-r--r-- 1 root root  1212 May 11 10:07 test.sa
-rw-r--r-- 1 root root  1213 May 11 10:07 test.vol


In [43]:
#@title Create reward using Zeo++
# both GSA (Gravimetric) and VSA (Volume) surface area
# probe radius of 1.525 Å (CO2 length is 3.5Å) and 2000 samples
class SurfaceAreaRewarder():
    def __init__(self, builder,
                 gsa_cutoff, vsa_cutoff,
                 temporary_dir = "scratch"):
        self.builder = builder
        self.gsa_cutoff = gsa_cutoff
        self.vsa_cutoff = vsa_cutoff
        self.temporary_dir = temporary_dir
        os.makedirs(temporary_dir, exist_ok=True)

    def reward(self, sequence):
        '''
        Gravimetric surface area is ASA (accessible area) + NASA
        Volume surface area is AV (accessible volume) + NAV
        '''
        with tempfile.NamedTemporaryFile(suffix=".cif", dir=self.temporary_dir, delete=True) as temp:
            mof = builder.make_pormake_mof(sequence)
            mof.write_cif(temp.name)

            command_1=(['./zeo++-0.3/network']  + ['-sa'] + ['1.525'] + ['1.525'] + ['2000'] + [temp.name])
            subprocess.run(command_1,stdout = subprocess.DEVNULL)
            command_2=(['./zeo++-0.3/network']  + ['-vol'] + ['1.525'] + ['1.525'] + ['2000'] + [temp.name])
            subprocess.run(command_2,stdout = subprocess.DEVNULL)
            output_sa = temp.name.removesuffix(".cif") + ".sa"
            output_vol = temp.name.removesuffix(".cif") + ".vol"

            def genric_reward(output_filename):
              output_reward = 0
              if os.path.exists(output_filename) == False:
                  return output_reward
              # else calculate
              with open(output_filename) as result_file:
                  lines=[]
                  for line in result_file:
                      lines.append(line.rstrip())
                  # Zeo++ writes empty output file for some non-physical MOFs. Reward them as 0.
                  if len(lines)==0:
                      output_reward = 0
                  else:
                      frags=lines[0].split()
                      NASA_or_NAV=float(frags[17])
                      ASA_or_AV=float(frags[11])
                      # Gravimetric_or_Volume surface area is NASA_or_NAV + ASA_or_AV
                      output_reward = ASA_or_AV + NASA_or_NAV
                      #output_reward = math.exp((total_GSA_or_VSA - gsa_or_vsa_cutoff) / gsa_or_vsa_cutoff)
                  # Remove zeo++ output file
                  # Temp file will be removed by the surrounding "with"
                  os.remove(output_filename)
              return output_reward
            # calculate
            total_cost_GSA_plus_VSA = genric_reward(output_sa) + genric_reward(output_vol)
            cutoff = self.gsa_cutoff + self.vsa_cutoff
            output_reward = math.exp((total_cost_GSA_plus_VSA - cutoff) / cutoff)
            # step function
            output_reward = output_reward if output_reward > 0 else 0
            return output_reward

In [44]:
rewarder = SurfaceAreaRewarder(builder,
                               gsa_cutoff = 5000, vsa_cutoff= 2,
                               temporary_dir = "scratch")


### Test reward function

In [45]:
sequence = builder.random_sequence()
print(sequence, rewarder.reward(sequence))

/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)


['N387', 'N355', 'N519', 'E86', 'E114'] 1.2813204317895999


In [46]:
env = SequenceEnvironment(
    token_vocabulary = builder.token_vocabulary,
    termination_token = builder.termination_token,
    reward_function = lambda s: rewarder.reward(s),
    render_function = None,
    mask = builder.mask,
    max_sequence_length = builder.n_slots, min_sequence_length = builder.n_slots
    )

flow_model = LSTM(token_vocabulary = builder.token_vocabulary, n_actions = env.action_space.n)
agent = TrajectoryBalanceGFlowNet(env, flow_model)

### train
#### learning rate of $5 × 10^3$
#### For demo : episodes = 1000

In [51]:
agent.train(True)
observations, infos, rewards, losses, logZs = agent.fit(learning_rate=5e-3, num_episodes=1000, minibatch_size=5)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
  0%|          | 1/1000 [00:03<51:38,  3.10s/it]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
  0%|          | 2/1000 [00:05<44:13,  2.66s/it]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
  0%|          | 3/1000 [00:08<46:38,  2.81s/it]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets o

## Generate 10 observations

In [52]:
agent.eval()
trained_observations = []

for i in tqdm(range(0, 10)):
    obs, info, reward, _ = agent.sample()
    sequence = agent.env._sequence[:-1]
    trained_observations.append([sequence, 0 if reward == 0 else math.log(reward) * cutoff + cutoff])

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
 10%|█         | 1/10 [00:04<00:43,  4.81s/it]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
 20%|██        | 2/10 [00:11<00:49,  6.19s/it]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
 30%|███       | 3/10 [00:16<00:37,  5.34s/it]/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vector

In [58]:
trained_observations

[[['N288', 'N424', 'N19', 'E32', 'E148'], 8111.821171531388],
 [['N263', 'N549', 'N353', 'E170', 'E84'], 6048.799540183927],
 [['N387', 'N718', 'N137', 'E166', 'E92'], 4679.079528188725],
 [['N524', 'N631', 'N535', 'E3', 'E3'], 8027.998600559777],
 [['N128', 'N551', 'N333', 'E104', 'E89'], 5900.448120751698],
 [['N603', 'N207', 'N410', 'E118', 'E139'], 5581.801879248301],
 [['N44', 'N302', 'N80', 'E199', 'E110'], 6910.134746101559],
 [['N335', 'N153', 'N495', 'E11', 'E230'], 4812.632217113155],
 [['N42', 'N257', 'N579', 'E16', 'E102'], 3988.046181527389],
 [['N688', 'N128', 'N353', 'E222', 'E89'], 4715.671091563375]]

In [59]:
structure =  builder.make_structure(trained_observations[0][0])

# Visualise it with Pymatgen
nv.show_pymatgen(structure)

/usr/local/lib/python3.11/dist-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)


NGLWidget()